In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import datasets
from sklearn.metrics import accuracy_score
import timm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(60),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
dataset = datasets.ImageFolder(r'C:\Users\Saumya\PycharmProjects\pythonProject6\data\AugmentedAlzheimerDataset',transform=train_transform)


In [5]:
class_counts = {}
subset_indices = []

for idx, (_, label) in enumerate(dataset.samples):  
    if class_counts.get(label, 0) < 4000:  
        subset_indices.append(idx)
        class_counts[label] = class_counts.get(label, 0) + 1

# Create a subset dataset
subset_dataset = Subset(dataset, subset_indices)


train_size = int(0.8 * len(subset_indices))  # 80% train
test_size = len(subset_indices) - train_size  # 20% test

# Split the subset dataset
train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])
train_val_split = int(0.8 * train_size)  
val_size = train_size - train_val_split
train_dataset, val_dataset = random_split(train_dataset, [train_val_split, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
testing_dataset = datasets.ImageFolder(r'C:\Users\Saumya\PycharmProjects\pythonProject6\OriginalDataset',transform=test_transform)

In [7]:
final_test_loader = DataLoader(testing_dataset, batch_size=32, shuffle=False)

In [8]:
class_mapping = {
    "Mild Impairment":"MildDemented",
    "Moderate Impairment":"ModeratedDemented",
    "No Impairment":"NonDemented",
    "Very Mild Impairment":"VeryMildDemented"
}

In [9]:
new_test_dataset = datasets.ImageFolder(r"C:\Users\Saumya\PycharmProjects\pythonProject6\test",transform=test_transform)
new_test_dataset.class_to_idx = {class_mapping[c]: idx for c, idx in new_test_dataset.class_to_idx.items() if c in class_mapping}

In [10]:
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)

In [11]:
from timm import create_model

model = create_model('edgenext_small.usi_in1k', pretrained=True)
print(model)

EdgeNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 48, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((48,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): EdgeNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvBlock(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
          (norm): LayerNorm((48,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=48, out_features=192, bias=True)
            (act): GELU(approximate='none')
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=192, out_features=48, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvBlock(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
          (norm): Layer

In [12]:
print(model)

EdgeNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 48, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((48,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): EdgeNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvBlock(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
          (norm): LayerNorm((48,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=48, out_features=192, bias=True)
            (act): GELU(approximate='none')
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=192, out_features=48, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvBlock(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
          (norm): Layer

In [13]:

old_head = model.head
num_features = model.head.fc.out_features  
num_classes = len(dataset.classes)
model.fc = nn.Sequential(
    old_head,
    nn.Linear(num_features, num_classes)
)
model = model.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [15]:
from sklearn.metrics import accuracy_score

def evaluate_model(model, val_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # No need to track gradients
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move to correct device
            outputs = model(images)  # Get model predictions
            _, preds = torch.max(outputs, 1)  # Get predicted class index

            all_labels.extend(labels.cpu().numpy())  # Store true labels
            all_preds.extend(preds.cpu().numpy())  # Store predictions

    accuracy = accuracy_score(all_labels, all_preds) * 100  
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    return accuracy  
def train_model(model, train_loader,val_loader, optimizer, criterion, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
    
    
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            train_loss += loss.item()
            
        train_loss /= len(train_loader)

    
    
        val_acc=evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {val_acc:.2f}%")


In [16]:
for param in model.parameters():
    param.requires_grad = True

In [17]:
print("\n Training only the classifier (Feature Extraction Mode)...")
train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=20,device=device)


 Training only the classifier (Feature Extraction Mode)...
Test Accuracy: 68.48%
Epoch 1/20, Loss: 0.7208, Accuracy: 68.48%
Test Accuracy: 73.87%
Epoch 2/20, Loss: 0.3778, Accuracy: 73.87%
Test Accuracy: 80.31%
Epoch 3/20, Loss: 0.4576, Accuracy: 80.31%
Test Accuracy: 81.02%
Epoch 4/20, Loss: 0.2619, Accuracy: 81.02%
Test Accuracy: 85.90%
Epoch 5/20, Loss: 0.1222, Accuracy: 85.90%
Test Accuracy: 87.42%
Epoch 6/20, Loss: 0.3259, Accuracy: 87.42%
Test Accuracy: 87.81%
Epoch 7/20, Loss: 0.1343, Accuracy: 87.81%
Test Accuracy: 94.18%
Epoch 8/20, Loss: 0.0422, Accuracy: 94.18%
Test Accuracy: 92.81%
Epoch 9/20, Loss: 0.0847, Accuracy: 92.81%
Test Accuracy: 95.51%
Epoch 10/20, Loss: 0.0576, Accuracy: 95.51%
Test Accuracy: 95.66%
Epoch 11/20, Loss: 0.1586, Accuracy: 95.66%
Test Accuracy: 96.91%
Epoch 12/20, Loss: 0.0524, Accuracy: 96.91%
Test Accuracy: 93.52%
Epoch 13/20, Loss: 0.0533, Accuracy: 93.52%
Test Accuracy: 96.33%
Epoch 14/20, Loss: 0.0134, Accuracy: 96.33%
Test Accuracy: 96.29%
Epo

In [18]:
optimzer = optim.Adam(model.parameters(), lr=1e-6)
print("\n Training only the classifier (Feature Extraction Mode)...")
train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10,device=device)


 Training only the classifier (Feature Extraction Mode)...
Test Accuracy: 97.81%
Epoch 1/10, Loss: 0.0471, Accuracy: 97.81%
Test Accuracy: 96.76%
Epoch 2/10, Loss: 0.0304, Accuracy: 96.76%
Test Accuracy: 98.01%
Epoch 3/10, Loss: 0.0059, Accuracy: 98.01%
Test Accuracy: 98.36%
Epoch 4/10, Loss: 0.0024, Accuracy: 98.36%
Test Accuracy: 97.89%
Epoch 5/10, Loss: 0.0278, Accuracy: 97.89%
Test Accuracy: 98.12%
Epoch 6/10, Loss: 0.0021, Accuracy: 98.12%
Test Accuracy: 97.73%
Epoch 7/10, Loss: 0.0179, Accuracy: 97.73%
Test Accuracy: 96.17%
Epoch 8/10, Loss: 0.0482, Accuracy: 96.17%
Test Accuracy: 98.20%
Epoch 9/10, Loss: 0.0027, Accuracy: 98.20%
Test Accuracy: 98.44%
Epoch 10/10, Loss: 0.0102, Accuracy: 98.44%


In [22]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score

model.eval()
all_labels = []
all_preds = []
val_loss = 0.0

with torch.no_grad():  # No need to track gradients for testing
    for images, labels in final_test_loader:
        images, labels = images.to(device), labels.to(device)  # Move labels and images to the current device
        outputs = model(images)  # Get model predictions
        _, preds = torch.max(outputs, 1)  # Get predicted class index (highest probability)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        all_labels.extend(labels.cpu().numpy())  # Store true labels
        all_preds.extend(preds.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)

# Compute F1-score
f1 = f1_score(all_labels, all_preds, average="weighted")  # Use "macro" for equal class weighting

precision = precision_score(all_labels, all_preds, average="weighted")  # Use "micro", "macro", or "weighted"

# Compute recall
recall = recall_score(all_labels, all_preds, average="weighted")
# Print results
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {val_loss:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# P
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))


Test Accuracy: 98.98%
Test Loss: 5.9984
F1 Score: 0.9898
Precision: 0.9899
Recall: 0.9898

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       896
           1       1.00      1.00      1.00        64
           2       0.99      0.99      0.99      3200
           3       0.98      0.99      0.99      2240

    accuracy                           0.99      6400
   macro avg       0.99      0.99      0.99      6400
weighted avg       0.99      0.99      0.99      6400



In [23]:
torch.save(model,"edgenext_small_usi_in1k_finetuned.pth")